In [ ]:
pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas

In [2]:
import cv2
import csv
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from pathlib import Path

# Video path and output CSV path
video_path = 'data/videos/test0_toy_lecture.mp4'
output_csv = 'annotations.csv'
sampling_frequency = 1  # seconds between samples

# Prepare CSV file
if not Path(output_csv).exists():
    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Frame', 'Count', 'Scene change'])  # Headers: frame number and label

# Load video
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_increment = int(fps * sampling_frequency)

# Widgets
button_change = widgets.Button(description="Scene change")
button_no_change = widgets.Button(description="No scene change")

# Initialize frame position to the last frame minus increment
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frame = int(total_frames - (total_frames % fps) - frame_increment)
count = 0
 

# Define button callback functions
def log_and_next(label):
    global frame
    global count
    # Log the label and frame number
    with open(output_csv, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([frame, count, label])
        count += 1

    # Move to next frame
    display_buttons()
    display_frame()

def on_change_button_clicked(b):
    log_and_next('True')

def on_no_change_button_clicked(b):
    log_and_next('False')

# Attach event handlers
button_change.on_click(on_change_button_clicked)
button_no_change.on_click(on_no_change_button_clicked)

# Display frame
def display_frame():
    global frame

    # Check for end of video
    if frame < 0:
        print("End of Video")
        cap.release()
        return

    # Set current frame
    frame -= frame_increment  # Decrement frame by frame_increment
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
    ret_current, current_frame_image = cap.read()
    if not ret_current:
        print("Failed to load current frame")
        return
    
    # Set last frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame + frame_increment)
    ret_last, last_frame_image = cap.read()
    if not ret_last:
        print("Failed to load last frame")
        return

    # Convert frames to RGB for display
    current_frame_rgb = cv2.cvtColor(current_frame_image, cv2.COLOR_BGR2RGB)
    last_frame_rgb = cv2.cvtColor(last_frame_image, cv2.COLOR_BGR2RGB)

    # Display frames using ipywidgets.Image
    print("Scene change if the top frame is different from the bottom frame")
    print("Top image frame: " + str(frame))
    current_frame_widget = widgets.Image(
        value=cv2.imencode('.jpg', current_frame_rgb)[1].tobytes(),
        format='jpg',
        width='45%',
        height='45%',
        min_width = '300px'
    )
    prev_frame = frame + frame_increment
    print("Bottom image frame: " + str(prev_frame))
    last_frame_widget = widgets.Image(
        value=cv2.imencode('.jpg', last_frame_rgb)[1].tobytes(),
        format='jpg',
        width='45%',
        height='45%',
    )

    display(current_frame_widget, last_frame_widget)

# Display buttons
def display_buttons():
    clear_output(wait=True)
    display(widgets.HBox([button_change, button_no_change]))

# Start the frame display
display_buttons()
display_frame()

Scene change if the top frame is different from the bottom frame
Top image frame: 875
Bottom image frame: 900


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…